In [9]:
import cv2
import os
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm

In [10]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [11]:
class image_mask():
    def __init__(self, csv_file, output_folder, idx, img_name, patch_size, stride, point, new_csv):
        self.data = pd.read_csv(csv_file)
        self.output_folder = output_folder
        self.idx = idx
        self.img_name = img_name
        self.patch_size = patch_size
        self.stride = stride
        self.point = point
        self.new_csv = new_csv
        
    def start(self):
        global count
        img_path = self.data.iloc[self.idx, 1]
        image = cv2.imread(img_path)
        
        mask_rle = self.data.iloc[self.idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
        
        #검증을 위한 분리
        for y in range(0, self.point + 1, self.stride):
            for x in range(0, self.point + 1, self.stride):
                
                #이미지, 마스크 자르기
                patch_image = image[y:y+self.patch_size, x:x+self.patch_size]
                patch_mask = mask[y:y+self.patch_size, x:x+self.patch_size]
                
                #이미지 파일에 이미지 저장
                patch_filename = f"{self.output_folder}/{self.img_name}{count}.png"
                cv2.imwrite(patch_filename, patch_image)
                patch_mask = rle_encode(patch_mask)
                
                #csv파일에 이미지 이름,이미지 위치,마스크값 저장
                if patch_mask == "":
                    patch_mask = -1

                csvwriter = csv.writer(self.new_csv)
                csvwriter.writerow([f'{self.img_name}{count}', f'./{self.output_folder}/{self.img_name}{count}.png', patch_mask])
                
                count+=1    

In [12]:
if __name__ == "__main__":
    #타일링할 이미지 이름, 위치, mark이 들어간 csv파일
    csv_file = "./train.csv"
    #타일링한 이미지를 넣을 파일
    output_folder = "train_ene"
    #생성될 이름 이름_(0부터 csv파일 길이 끝까지 숫자로 작됨)
    img_name = "TRAIN_endT_"
    #생성할 csv이름
    csv_name = "train_end"
    #이미지 가로,세로 크기 정하기
    patch_size=256
    #stride 픽셀만큼 움직임
    stride = 192
    #point 픽셀까지 움직임
    point = 768
    
    global count
    count = 0
    idx = 0
    csv_len = pd.read_csv(csv_file)
    new_csv = open(f'{csv_name}.csv', 'w', newline='')
    csvwriter = csv.writer(new_csv)
    csvwriter.writerow(['img_id', 'img_path', 'mask_rle'])
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for idx in tqdm(range(len(csv_len))):
        image_mask_instance = image_mask(csv_file, output_folder, idx, img_name, patch_size, stride, point, new_csv)
        image_mask_instance.start()
    
    new_csv.close()

100%|██████████| 6426/6426 [1:51:21<00:00,  1.04s/it]
